# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_csv = pd.read_csv("../WeatherPy/output.csv")
weather_csv

,City,Lat,Lng,Temperature Max,Humidity,Clouds,Wind Speed,Country,Date
0,Busselton,-33.6500,115.3333,50.09,77,92,4.70,AU,1635027095
1,Jamestown,42.0970,-79.2353,46.29,97,90,1.99,US,1635027096
2,Ushuaia,-54.8000,-68.3000,33.46,93,75,18.41,AR,1635027096
3,Erdemli,36.6050,34.3084,70.21,38,0,3.27,TR,1635027096
4,Rikitea,-23.1203,-134.9692,74.88,82,97,20.09,PF,1635027097
...,...,...,...,...,...,...,...,...,...
589,Tra Vinh,9.9347,106.3453,76.14,91,83,3.42,VN,1635027294
590,Bamnet Narong,15.4992,101.6897,69.06,92,82,9.24,TH,1635027294
591,Kulachi,31.9286,70.4592,66.79,68,0,4.32,PK,1635027295
592,Oktyabr'skoye,43.0645,44.7417,32.14,100,0,6.71,RU,1635027295


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_csv[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_csv["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure()
max_intensity = np.max(humidity)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 70 and 80
reduced_df = weather_csv.loc[(weather_csv["Wind Speed"] <= 10) & (weather_csv["Clouds"] == 0) & \
                                   (weather_csv["Temperature Max"] >= 70) & (weather_csv["Temperature Max"] <= 80)].dropna()

reduced_df

,City,Lat,Lng,Temperature Max,Humidity,Clouds,Wind Speed,Country,Date
3,Erdemli,36.6050,34.3084,70.21,38,0,3.27,TR,1635027096
33,Cayenne,4.9333,-52.3333,78.10,88,0,0.00,GF,1635027107
65,Avera,33.1940,-82.5271,78.31,48,0,2.80,US,1635027116
162,Dondo,-19.6094,34.7431,73.18,84,0,4.03,MZ,1635027149
226,Inverell,-29.7833,151.1167,74.66,56,0,8.99,AU,1635027170
241,Kiryat Gat,31.6100,34.7642,70.81,100,0,4.25,IL,1635027175
258,Salimpur,28.1280,76.2766,71.85,60,0,5.68,IN,1635027180
314,Basi,26.8333,76.0333,71.94,69,0,3.47,IN,1635027201
361,Samandağ,36.0842,35.9771,70.32,69,0,4.36,TR,1635027218
362,Havelock,34.8791,-76.9013,73.96,71,0,3.94,US,1635027218


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = reduced_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
3,Erdemli,TR,36.6050,34.3084,
33,Cayenne,GF,4.9333,-52.3333,
65,Avera,US,33.1940,-82.5271,
162,Dondo,MZ,-19.6094,34.7431,
226,Inverell,AU,-29.7833,151.1167,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [9]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field.. skipping.")
        
    print("------------")

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 3: Erdemli.
Closest hotel in Erdemli is Denizkumu Hotel.
------------
Retrieving Results for Index 33: Cayenne.
Closest hotel in Cayenne is Grand Hotel Montabo.
------------
Retrieving Results for Index 65: Avera.
Missing field.. skipping.
------------
Retrieving Results for Index 162: Dondo.
Missing field.. skipping.
------------
Retrieving Results for Index 226: Inverell.
Closest hotel in Inverell is Imperial Hotel.
------------
Retrieving Results for Index 241: Kiryat Gat.
Closest hotel in Kiryat Gat is Desert Gat.
------------
Retrieving Results for Index 258: Salimpur.
Missing field.. skipping.
------------
Retrieving Results for Index 314: Basi.
Missing field.. skipping.
------------
Retrieving Results for Index 361: Samandağ.
Closest hotel in Samandağ is Truva Life Hotel.
------------
Retrieving Results for Index 362: Havelock.
Closest hotel in Havelock is Hampton Inn Havelock.
------------
Retrieving Results for Index 373: Jiwani.
Closest hotel in J

In [10]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
3,Erdemli,TR,36.6050,34.3084,Denizkumu Hotel
33,Cayenne,GF,4.9333,-52.3333,Grand Hotel Montabo
65,Avera,US,33.1940,-82.5271,
162,Dondo,MZ,-19.6094,34.7431,
226,Inverell,AU,-29.7833,151.1167,Imperial Hotel
241,Kiryat Gat,IL,31.6100,34.7642,Desert Gat
258,Salimpur,IN,28.1280,76.2766,
314,Basi,IN,26.8333,76.0333,
361,Samandağ,TR,36.0842,35.9771,Truva Life Hotel
362,Havelock,US,34.8791,-76.9013,Hampton Inn Havelock


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))